In [1]:
from matplotlib import pyplot as plt
from matplotlib.pyplot import imread
import os 
import numpy as np
from skimage.color import rgb2hsv
import math
from skimage import feature, io

In [2]:
def rgb2gray(rgb):
    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b
    return gray

def textureFeatures(img):
    img=np.array(rgb2gray(img), int)
    glcm=feature.greycomatrix(img,distances=[0,1], angles=[0, np.pi/4, np.pi/2, 3*np.pi/4],levels=255 , symmetric=True, normed=True)
    texture_features=np.concatenate((feature.greycoprops(glcm, 'contrast'),feature.greycoprops(glcm, 'correlation'),feature.greycoprops(glcm, 'energy'),feature.greycoprops(glcm, 'homogeneity')))
    texture_features=texture_features/np.sum(texture_features)
    return texture_features

# a=imread("DataToPredict/535099.jpg")
# print(textureFeatures(a).flatten())
# help(feature.greycoprops)

In [3]:
def hsvHistogramFeatures(img):
    [rows,cols,numOfBands]=img.shape
    
    numberOfLevelsForH = 8; # 8 niveau pour h
    numberOfLevelsForS = 2; # 2 niveau pour s
    numberOfLevelsForV = 2; # 2 niveau pour v
    
    img=rgb2hsv(img)
    h=img[:,:,0]
    s=img[:,:,1]
    v=img[:,:,2]
    maxValueForH=h.max()
    maxValueForS=s.max()
    maxValueForV=v.max()
    hsvColor_Histogram=np.zeros((8,2,2))
    
    quantizedValueForH= np.ceil((numberOfLevelsForH * h)/maxValueForH);
    quantizedValueForS= np.ceil((numberOfLevelsForS * s)/maxValueForS);
    quantizedValueForV= np.ceil((numberOfLevelsForV * v)/maxValueForV);
    index=np.zeros((rows*cols,3))
    index[:,0]=quantizedValueForH.T.reshape(1,-1)
    index[:,1]=quantizedValueForS.T.reshape(1,-1)
    index[:,2]=quantizedValueForV.T.reshape(1,-1)

    for row in range(0,index.shape[0]):
        if(index[row][0]==0 or index[row][1]==0 or index[row][2]==0):
            continue
        hsvColor_Histogram[int(index[row][0])-1][int(index[row][1])-1][int(index[row][2])-1]=hsvColor_Histogram[int(index[row][0])-1][int(index[row][1])-1][int(index[row][2])-1]+1
     
    hsvColor_Histogram=hsvColor_Histogram.T
    hsvColor_Histogram=hsvColor_Histogram/np.sum(hsvColor_Histogram)
    return hsvColor_Histogram

# a=imread('DataToPredict/535099.jpg')
# print(hsvHistogramFeatures(a).flatten())
# print(list(hsvHistogramFeatures(a).reshape(1,-1)))

In [4]:
def color_Moments(img):
    img=rgb2hsv(img)
    
    R=img[:,:,0]
    G=img[:,:,1]
    B=img[:,:,2]
    
    colorFeatures=[np.mean(R[:]),np.std(R[:]),np.mean(G[:]),np.std(G[:]),np.mean(B[:]),np.std(B[:])]
    colorFeatures=colorFeatures/np.mean(colorFeatures)
    return colorFeatures
          
#a=imread("DataToPredict/535099.jpg")
#color_Moments(a)

In [5]:
def getFeatures(img,fsize):
    features=np.zeros((fsize-1,1))
    features=color_Moments(img)
    features=np.concatenate((features,hsvHistogramFeatures(img).flatten()))#,textureFeatures(img).flatten()
    return features

# img=imread('DataToPredict/535099.jpg')
# print(getFeatures(img,50).shape)

In [6]:
def createFeatures(fold):
    features=np.zeros((490,38))
    etat=[]
    list=os.listdir(fold)
    k=0
    for i in range(0,len(list)):
        if(os.path.isdir(fold+"/"+list[i])):
            sub=os.listdir(fold+"/"+list[i])
            for j in range(0,len(sub)):
                etat.append(list[i])
                IDB=imread(fold+"/"+list[i]+"/"+sub[j])
                features[k][:]=getFeatures(IDB,38)
                k=k+1
    return etat,features

e,f=createFeatures('DB2C')

In [7]:
from sklearn.model_selection import train_test_split
import random
from sklearn import tree
from sklearn import datasets
import csv
clf=tree.DecisionTreeClassifier()
features = np.array(f)
etats = np.array(e)
(trainFeat, testFeat, trainEtat, testEtat) = train_test_split(features, etats, test_size=0.1, random_state=random.seed())
clf.fit(trainFeat,trainEtat)
print("scope estime du classifieur appris",clf.score(testFeat,testEtat))
def Prediction(path):
    list=os.listdir(path)
    for i in range(0,len(list)):
        IPB=imread(path+"/"+list[i])
        features=getFeatures(IPB,38)
        with open('Prediction.csv', mode='a') as file:
            writer = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writer.writerow([list[i], clf.predict([features])])
        print(list[i],clf.predict([features]))
        
Prediction("DataToPredict")

scope estime du classifieur appris 0.8979591836734694
521079.jpg ['obj_car']
447063.jpg ['obj_car']
521022.jpg ['obj_car']
535095.jpg ['obj_car']
535091.jpg ['obj_ship']
447012.jpg ['obj_car']
535015.jpg ['obj_ship']
521090.jpg ['obj_car']
535010.jpg ['obj_car']
535031.jpg ['obj_ship']
535099.jpg ['obj_car']
29055.jpg ['obj_car']
535000.jpg ['obj_ship']
535082.jpg ['obj_ship']
29011.jpg ['obj_car']
354001.jpg ['obj_car']
535041.jpg ['obj_ship']
29013.jpg ['obj_car']
354053.jpg ['obj_car']
535078.jpg ['obj_car']


In [8]:
from sklearn import svm
(trainFeat, testFeat, trainEtat, testEtat) = train_test_split(features, etats, train_size=0.9, test_size=0.1,random_state=random.seed())
clf = svm.SVC(gamma=0.01, C=100)
clf.fit(trainFeat,trainEtat)#entrainemant de clf sur l'échantillon d'apprentissage
print("taux de précision",clf.score(testFeat,testEtat))

taux de précision 0.9795918367346939
